In [101]:
import numpy as np
import pandas as pd

In [102]:
df=pd.read_excel('test2.xlsx')

In [103]:
df['date'] = pd.to_datetime(df['date'])

In [104]:
df['year_sold']=df['date'].dt.year

In [105]:
df['HouseAge'] = df['year_sold'] - df['yr_built']

In [106]:
from scipy.stats import skew
import numpy as np

# numeric columns only
numeric_cols = df.select_dtypes(include=np.number)

# compute skew
skewness = numeric_cols.apply(lambda x: skew(x.dropna()))

# select highly skewed features
skewed_feats = skewness[skewness > 0.75].index

# keep ONLY columns that are strictly non-negative
log_safe_feats = [
    col for col in skewed_feats
    if (df[col] >= 0).all()
]

log_safe_feats


['sqft_living',
 'sqft_lot',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_renovated',
 'sqft_living15',
 'sqft_lot15']

In [107]:
df[log_safe_feats] = np.log1p(df[log_safe_feats])


In [108]:
tabular_features = [
    col for col in df.columns
    if col not in [
        "date",
        "sqft_living",
        "yr_renovated",
        "long",
        "year_sold",
        "id",
    ]
]

X_pred = df[tabular_features]

In [109]:
X_pred.head()

,bedrooms,bathrooms,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,zipcode,lat,sqft_living15,sqft_lot15,HouseAge
0,4,2.25,9.093132,2.0,0.0,0.000000,1.609438,2.197225,7.635787,0.000000,1986,98058,47.4388,7.779467,8.949105,28
1,5,3.00,8.814479,1.0,0.0,0.000000,1.791759,2.197225,7.512618,6.976348,1977,98115,47.6784,7.771067,8.745762,37
2,4,2.50,9.295967,2.0,0.0,1.098612,1.386294,2.484907,8.235095,0.000000,1997,98006,47.5646,8.219057,9.178437,17
3,3,3.50,9.589393,2.0,0.0,1.098612,1.386294,2.564949,8.425297,0.000000,1990,98034,47.6995,8.306719,9.562897,25
4,3,2.50,8.589886,2.0,0.0,0.000000,1.386294,2.302585,7.844241,0.000000,2004,98052,47.6647,7.719130,8.306719,10


In [110]:
import joblib

# Load model
lgbm_loaded = joblib.load("lgbm_tabular_model.pkl")

In [111]:
# Predict (log-scale)
y_pred_log = lgbm_loaded.predict(X_pred)

# 🔥 Inverse log transform to get REAL prices
y_pred_original = np.expm1(y_pred_log)

# Create submission
submission = pd.DataFrame({
    "id": df["id"],                 # same order, unchanged
    "predicted_price": y_pred_original
})

submission.to_csv("submission.csv", index=False)

print("✅ submission.csv created successfully (original scale)")
print(submission.head())


✅ submission.csv created successfully (original scale)
           id  predicted_price
0  2591820310     5.592406e+05
1  7974200820     1.810729e+06
2  7701450110     2.122442e+06
3  9522300010     2.236359e+06
4  9510861140     1.266722e+06
